In [10]:
!git clone https://github.com/Muennighoff/transformers.git

!cd trans*; pip install -q -e '.[dev]'
%cd transformers/src

!pip install -q datasets
!pip install -q bleach

fatal: destination path 'transformers' already exists and is not an empty directory.
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
/content/transformers/src


##### Activate TPU (only for TF)

In [14]:
# If this cell takes very long to run; factory reset and run this cell the very first
import tensorflow as tf
import os

tf.keras.backend.clear_session()

try:
    # Locate TPU
    TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    # Establish communication / Locates TPU on the network
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER)
    # Connect
    tf.config.experimental_connect_to_cluster(resolver)
    # Start TPU
    tf.tpu.experimental.initialize_tpu_system(resolver)
    # Set strategy
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

    print("Number of accelerators: ", strategy.num_replicas_in_sync)

except ValueError:
    print("No TPU found")

# Check devices
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.52.149.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.52.149.10:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of accelerators:  8
All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


##### Preprocessing

In [15]:
from datasets import load_dataset
from transformers import BertTokenizerFast

import numpy as np
import bleach
import re

import torch

In [16]:
dataset = load_dataset('hyperpartisan_news_detection', 'byarticle')
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'hyperpartisan', 'url', 'published_at'],
        num_rows: 645
    })
})

In [17]:
def clean_text(text, label):
    """ Clean the input text and remove special characters """
    text = bleach.clean(text,strip=True)
    text = text.replace('<p>', '')
    text = text.replace('</p>', '')
    text = text.replace('\n', '')
    text = text.replace('&amp;#160;', '')
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    if str(label) == 'True':
        new_label = 1
    else:
        new_label = 0
    return text, new_label


def convert_to_features(example):
    # Tokenize contexts and questions (as pairs of inputs)
    text_, target_ = clean_text(example['text'], example['hyperpartisan'])
    encodings = tokenizer.encode_plus(text_, pad_to_max_length=True, max_length=2048,
                                      add_special_tokens=True,
                                      return_token_type_ids=False,
                                      return_attention_mask=True,
                                      padding='max_length', truncation=True,
                                      )
    targets = torch.tensor(target_,dtype=torch.long)
    
    encodings.update({'labels': targets,
                      'attention_mask': encodings['attention_mask']})
    return encodings

In [18]:
## Take subset of data
#train_size = 20000
#val_size = 5000

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

#train_indices = np.random.randint(0, len(dataset['train']), train_size)
#val_indices = np.random.randint(0, len(dataset['validation']), val_size)
#train_dataset = dataset['train'].select(train_indices)
#val_dataset = dataset['validation'].select(val_indices)


train_dataset =  dataset['train'].select(range(0, 500)).map(convert_to_features, load_from_cache_file=False)
val_dataset =  dataset['train'].select(range(500, 600)).map(convert_to_features, load_from_cache_file=False)

In [19]:
# Not perfect cleaning but better than nothing for now
for i in dataset["train"].select(range(1)):
  print(i)
  print(i.keys())
  
for i in dataset["train"].select(range(1)):
  print(clean_text(i["text"], i["hyperpartisan"]))

{'hyperpartisan': True, 'published_at': '2017-09-10', 'text': '<p>Money ( <a href="https://farm8.static.flickr.com/7020/6551534889_9c8ae52997.jpg" type="external">Image</a> by <a href="https://www.flickr.com/people/68751915@N05/" type="external">401(K) 2013</a>) <a href="https://creativecommons.org/licenses/by-sa/2.0/" type="external">Permission</a> <a type="internal">Details</a> <a type="internal">DMCA</a></p> No Pill Can Stop Tinnitus, But This 1 Weird Trick Can \n<p>The walls are closing in on Congress.</p> \n<p>Terrifying walls of water from Hurricanes Harvey and Irma, which, when the damage is totaled, could rise to a half trillion dollars. The Walls of War: The multi-trillion dollar ongoing cost of Afghanistan, Iraq and other interventions. The crumbling walls of the U.S. infrastructure, which need at least $3 trillion to be repaired or replaced. A wall of 11 million undocumented immigrants, whose deportation could easily cost $200 billion. The planned wall at the Mexican border,

In [20]:
train_dataset

Dataset({
    features: ['attention_mask', 'hyperpartisan', 'input_ids', 'labels', 'published_at', 'text', 'title', 'url'],
    num_rows: 500
})

##### PyTorch

In [ ]:
columns = ['input_ids', 'attention_mask', 'labels']
train_dataset.set_format(type='torch', columns=columns)
val_dataset.set_format(type='torch', columns=columns)

In [10]:
import torch
import numpy as np
from transformers import BertForSequenceClassification, BertPerformerForSequenceClassification, BertPerformerConfig, BertConfig, Trainer, TrainingArguments

from datasets import load_metric
metric = load_metric("accuracy")

batch_size = 1
lr = 1e-5/8

args = TrainingArguments(
    "test",
    evaluation_strategy = "steps",
    eval_steps=100,
    logging_steps=100,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    print(predictions)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
config = BertPerformerConfig()

config.max_position_embeddings = 2048

model = BertPerformerForSequenceClassification(config)

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
100,1.351600,0.744218,0.770000,9.811500,10.192000
200,0.962800,1.196507,0.770000,9.659100,10.353000
300,1.876400,1.015505,0.770000,9.658500,10.354000
400,1.455100,1.060035,0.770000,9.658800,10.353000
500,1.806100,1.083990,0.770000,9.661400,10.350000
600,1.675300,1.039088,0.770000,9.663300,10.348000
700,1.767900,0.983283,0.770000,9.658600,10.353000
800,1.520200,1.069610,0.770000,9.660600,10.351000
900,1.440300,1.115328,0.770000,9.657900,10.354000
1000,1.769800,1.004918,0.770000,9.656200,10.356000


(100, 2) (100,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(100, 2) (100,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(100, 2) (100,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(100, 2) (100,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(100, 2) (100,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

KeyboardInterrupt: ignored

In [11]:
config = BertConfig()

config.max_position_embeddings = 2048

model = BertForSequenceClassification(config)

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
100,1.320000,0.730928,0.770000,12.890400,7.758000
200,0.945000,1.194015,0.770000,12.887500,7.759000
300,1.875600,1.001647,0.770000,12.885900,7.760000


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


KeyboardInterrupt: ignored

In [31]:
config = BertPerformerConfig.from_pretrained("bert-base-uncased")

model = BertPerformerForSequenceClassification.from_pretrained("bert-base-uncased", config=config)

keep_weights = True
if keep_weights:
    state_dict = model.state_dict()
    encoder_pos = state_dict['bert.embeddings.position_embeddings.weight']
    to_append = encoder_pos[2:]
    new_encoder_pos = torch.cat((encoder_pos, to_append, to_append, to_append, to_append[:6]))

model.bert.embeddings.position_embeddings = torch.nn.Embedding(2048, 768)

if keep_weights:
    state_dict['bert.embeddings.position_embeddings.weight'] = new_encoder_pos
    model.load_state_dict(state_dict)

model.bert.embeddings.position_ids = torch.arange(2048).expand((1, -1))

# Is it still a buffer?
for i in model.buffers():
  print(i)


trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertPerformerForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertPerformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertPerformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertPerformerForSequenceClassification were not init

tensor([[   0,    1,    2,  ..., 2045, 2046, 2047]])


Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
100,0.773500,0.581730,0.770000,9.804200,10.200000
200,0.595400,0.551279,0.770000,9.657000,10.355000
300,0.892200,0.537189,0.770000,9.658200,10.354000
400,0.836500,0.592185,0.770000,9.659300,10.353000
500,1.122900,0.716533,0.770000,9.660000,10.352000
600,1.133400,0.648080,0.770000,9.661800,10.350000
700,1.158000,0.643724,0.770000,9.660200,10.352000
800,1.115900,0.787727,0.770000,9.656100,10.356000
900,1.067100,0.848879,0.770000,9.661600,10.350000
1000,1.202700,0.782372,0.770000,9.657000,10.355000


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

KeyboardInterrupt: ignored

In [30]:
config = BertConfig.from_pretrained("bert-base-uncased")

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)

keep_weights = True
if keep_weights:
    state_dict = model.state_dict()
    encoder_pos = state_dict['bert.embeddings.position_embeddings.weight']
    to_append = encoder_pos[2:]
    new_encoder_pos = torch.cat((encoder_pos, to_append, to_append, to_append, to_append[:6]))

model.bert.embeddings.position_embeddings = torch.nn.Embedding(2048, 768)

if keep_weights:
    state_dict['bert.embeddings.position_embeddings.weight'] = new_encoder_pos
    model.load_state_dict(state_dict)

model.bert.embeddings.position_ids = torch.arange(2048).expand((1, -1))

# Is it still a buffer?
for i in model.buffers():
  print(i)


trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
100,0.752300,0.790338,0.240000,12.884200,7.761000
200,0.675000,0.542581,0.770000,12.883600,7.762000
300,0.709700,0.537961,0.740000,12.878700,7.765000
400,0.604200,0.507491,0.770000,12.881200,7.763000
500,0.566700,0.546589,0.740000,12.888900,7.759000
600,0.613500,0.500420,0.760000,12.881100,7.763000


[1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0
 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1
 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0

KeyboardInterrupt: ignored

##### TensorFlow

In [21]:
import tensorflow as tf

batch_size = 8
lr = 1e-5#/8

def to_tf(ds, seq_len):
    """
    Creates a TF Dataset given basic tokenizer() encoding
    """
    # Turn into TF Tensors
    ds.set_format('tensorflow', columns=['input_ids', 'attention_mask', 'labels'])
    # Extract TF Tensors
    features = {x: ds[x].to_tensor(default_value=0, shape=[None, seq_len]) for x in ['input_ids', 'attention_mask']}
    # Label is not a ragged tensor like the others & hence we separate it 
    tfds = tf.data.Dataset.from_tensor_slices((features, ds["labels"]))
    return tfds

train_ds = to_tf(train_dataset, 2048).shuffle(1000).batch(batch_size, drop_remainder=True)
val_ds = to_tf(val_dataset, 2048).batch(batch_size, drop_remainder=True)

In [22]:
from transformers import TFBertForSequenceClassification, TFBertPerformerForSequenceClassification, BertPerformerConfig, BertConfig


class WrapperFinetuning(tf.keras.Model):
    def __init__(self, model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = model
        # Track loss (Loss itself its CategoricalCrossEnt.)
        self.loss_tracker = tf.keras.metrics.Mean(name='loss') 

    @tf.function
    def train_step(self, data):
        # Data is a tuple of x, y for the CMNLI Finetuning data
        x, y = data
        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self.model(x, training=True).logits
            loss = self.compute_loss(y, y_pred)
            # Reduce loss to single digit
            loss = tf.reduce_mean(loss)
        # Compute gradients
        gradients = tape.gradient(loss, self.trainable_variables)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Update loss tracker
        self.loss_tracker.update_state(loss)  
        # Update metrics
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    @tf.function
    def test_step(self, data):
        x, y = data
        # Compute predictions
        y_pred = self.model(x, training=False).logits
        loss = self.compute_loss(y, y_pred)
        # Updates the metrics tracking the loss
        self.loss_tracker.update_state(loss)  
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    def compute_loss(self, labels, logits):
        if len(shape_list(logits)) == 1 or shape_list(logits)[1] == 1:
            loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
        else:
            # We use the below, as 3 labels
            loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits=True, reduction=tf.keras.losses.Reduction.NONE
            )

        return loss_fn(labels, logits)

def shape_list(tensor: tf.Tensor):
    """
    Deal with dynamic shape in tensorflow cleanly.
    Args:
        tensor (:obj:`tf.Tensor`): The tensor we want the shape of.
    Returns:
        :obj:`List[int]`: The shape of the tensor as a list.
    """
    dynamic = tf.shape(tensor)

    if tensor.shape == tf.TensorShape(None):
        return dynamic

    static = tensor.shape.as_list()

    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

In [23]:
for i in train_ds.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(8, 2048), dtype=int32, numpy=
array([[  101, 13857,  8112, ...,     0,     0,     0],
       [  101,  1004,  1001, ...,     0,     0,     0],
       [  101,  1006, 27166, ...,     0,     0,     0],
       ...,
       [  101,  2045,  2038, ...,     0,     0,     0],
       [  101, 18079,  1012, ...,  1010,  6699,   102],
       [  101,  2280, 11026, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(8, 2048), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}, <tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 1, 0, 1, 1, 1, 0, 0])>)


###### BERTPerformer

In [24]:
# Performer BS 8, Pretrained + copy over Pos

with strategy.scope():
    model_tf = TFBertPerformerForSequenceClassification.from_pretrained("bert-base-uncased")

    if True:
        # Reinitialize Pos Embeddings - This is necessary even on GPU if it throws no error as

        # On tf.gather:
        # Caution: On CPU, if an out of bound index is found, an error is raised. 
        # On GPU, if an out of bound index is found, a 0 is stored in the corresponding output value.

        print("Old pos:", tf.shape(model_tf.bert.embeddings.position_embeddings))
        encoder_pos = tf.identity(model_tf.bert.embeddings.position_embeddings)
        to_append = tf.identity(model_tf.bert.embeddings.position_embeddings[2:])
        new_encoder_pos = tf.concat((encoder_pos, to_append, to_append, to_append, to_append[:6]), axis=0)

        model_tf.bert.embeddings.position_embeddings = new_encoder_pos

        print("New pos", tf.shape(model_tf.bert.embeddings.position_embeddings))


    model_tf = WrapperFinetuning(model_tf)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model_tf.compile(optimizer=optimizer, metrics=['accuracy'])
    
model_tf.fit(train_ds, epochs=5, validation_data=val_ds)

All model checkpoint layers were used when initializing TFBertPerformerForSequenceClassification.


Some layers of TFBertPerformerForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Old pos: tf.Tensor([512 768], shape=(2,), dtype=int32)
New pos tf.Tensor([2048  768], shape=(2,), dtype=int32)
Epoch 1/5


62/62 [==============================] - ETA: 0s - accuracy: 0.5280 - loss: 0.6772WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


62/62 [==============================] - 94s 322ms/step - accuracy: 0.5285 - loss: 0.6772 - val_accuracy: 0.7604 - val_loss: 0.5508
Epoch 2/5
62/62 [==============================] - 10s 160ms/step - accuracy: 0.6939 - loss: 0.5796 - val_accuracy: 0.7604 - val_loss: 0.5261
Epoch 3/5
62/62 [==============================] - 10s 160ms/step - accuracy: 0.7482 - loss: 0.5183 - val_accuracy: 0.7188 - val_loss: 0.5421
Epoch 4/5
62/62 [==============================] - 10s 159ms/step - accuracy: 0.8468 - loss: 0.3796 - val_accuracy: 0.7917 - val_loss: 0.4287
Epoch 5/5
62/62 [==============================] - 10s 159ms/step - accuracy: 0.8607 - loss: 0.3126 - val_accuracy: 0.8333 - val_loss: 0.4273


In [22]:
# Performer BS 8 Pretrained

with strategy.scope():
    model_tf = TFBertPerformerForSequenceClassification.from_pretrained("bert-base-uncased")

    if True:
        # Reinitialize Pos Embeddings - This is necessary even on GPU if it throws no error as

        # On tf.gather:
        # Caution: On CPU, if an out of bound index is found, an error is raised. 
        # On GPU, if an out of bound index is found, a 0 is stored in the corresponding output value.

        from transformers.modeling_tf_utils import get_initializer

        config = model_tf.config
        model_tf.bert.embeddings.position_embeddings = model_tf.bert.embeddings.add_weight(
                                                        name="embeddings",
                                                        shape=[2048, 768],
                                                        initializer=get_initializer(initializer_range=config.initializer_range),
                                                        )

    model_tf = WrapperFinetuning(model_tf)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model_tf.compile(optimizer=optimizer, metrics=['accuracy'])
    
model_tf.fit(train_ds, epochs=5, validation_data=val_ds)

All model checkpoint layers were used when initializing TFBertPerformerForSequenceClassification.


Some layers of TFBertPerformerForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5


62/62 [==============================] - ETA: 0s - accuracy: 0.4936 - loss: 0.6538WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


62/62 [==============================] - 84s 297ms/step - accuracy: 0.4952 - loss: 0.6538 - val_accuracy: 0.7604 - val_loss: 0.5273
Epoch 2/5
62/62 [==============================] - 10s 159ms/step - accuracy: 0.6486 - loss: 0.5869 - val_accuracy: 0.6250 - val_loss: 0.5955
Epoch 3/5
62/62 [==============================] - 11s 179ms/step - accuracy: 0.7689 - loss: 0.4924 - val_accuracy: 0.7708 - val_loss: 0.4706
Epoch 4/5
62/62 [==============================] - 10s 159ms/step - accuracy: 0.8124 - loss: 0.4130 - val_accuracy: 0.7604 - val_loss: 0.4601
Epoch 5/5
62/62 [==============================] - 10s 160ms/step - accuracy: 0.7854 - loss: 0.4757 - val_accuracy: 0.7708 - val_loss: 0.5778


In [25]:
# Performer BS 8, Not Pretrained

with strategy.scope():
    config = BertPerformerConfig()
    config.max_position_embeddings = 2048
    model_tf = TFBertPerformerForSequenceClassification(config)

    if False:
        # Reinitialize Pos Embeddings - This is necessary even on GPU if it throws no error as

        # On tf.gather:
        # Caution: On CPU, if an out of bound index is found, an error is raised. 
        # On GPU, if an out of bound index is found, a 0 is stored in the corresponding output value.

        from transformers.modeling_tf_utils import get_initializer

        config = model_tf.config
        model_tf.bert.embeddings.position_embeddings = model_tf.bert.embeddings.add_weight(
                                                        name="embeddings",
                                                        shape=[2048, 768],
                                                        initializer=get_initializer(initializer_range=config.initializer_range),
                                                        )

    model_tf = WrapperFinetuning(model_tf)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model_tf.compile(optimizer=optimizer, metrics=['accuracy'])
    
model_tf.fit(train_ds, epochs=5, validation_data=val_ds)

Epoch 1/5


62/62 [==============================] - ETA: 0s - accuracy: 0.5222 - loss: 0.6974WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


62/62 [==============================] - 90s 301ms/step - accuracy: 0.5229 - loss: 0.6974 - val_accuracy: 0.7604 - val_loss: 0.5893
Epoch 2/5
62/62 [==============================] - 10s 157ms/step - accuracy: 0.6251 - loss: 0.6840 - val_accuracy: 0.7604 - val_loss: 0.5647
Epoch 3/5
62/62 [==============================] - 10s 158ms/step - accuracy: 0.5583 - loss: 0.6829 - val_accuracy: 0.7604 - val_loss: 0.5633
Epoch 4/5
62/62 [==============================] - 10s 159ms/step - accuracy: 0.5888 - loss: 0.6657 - val_accuracy: 0.7604 - val_loss: 0.5429
Epoch 5/5
62/62 [==============================] - 10s 161ms/step - accuracy: 0.6078 - loss: 0.6380 - val_accuracy: 0.7500 - val_loss: 0.4842


###### BERT

In [26]:
# BERT BS 8, Pretrained + copy over Pos

with strategy.scope():
    model_tf = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")

    if True:
        # Reinitialize Pos Embeddings - This is necessary even on GPU if it throws no error as

        # On tf.gather:
        # Caution: On CPU, if an out of bound index is found, an error is raised. 
        # On GPU, if an out of bound index is found, a 0 is stored in the corresponding output value.

        print("Old pos:", tf.shape(model_tf.bert.embeddings.position_embeddings))
        encoder_pos = tf.identity(model_tf.bert.embeddings.position_embeddings)
        to_append = tf.identity(model_tf.bert.embeddings.position_embeddings[2:])
        new_encoder_pos = tf.concat((encoder_pos, to_append, to_append, to_append, to_append[:6]), axis=0)

        model_tf.bert.embeddings.position_embeddings = new_encoder_pos

        print("New pos", tf.shape(model_tf.bert.embeddings.position_embeddings))


    model_tf = WrapperFinetuning(model_tf)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model_tf.compile(optimizer=optimizer, metrics=['accuracy'])
    
model_tf.fit(train_ds, epochs=5, validation_data=val_ds)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.


Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Old pos: tf.Tensor([512 768], shape=(2,), dtype=int32)
New pos tf.Tensor([2048  768], shape=(2,), dtype=int32)
Epoch 1/5


62/62 [==============================] - ETA: 0s - accuracy: 0.6327 - loss: 0.6000WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


62/62 [==============================] - 83s 368ms/step - accuracy: 0.6334 - loss: 0.6000 - val_accuracy: 0.6979 - val_loss: 0.5356
Epoch 2/5
62/62 [==============================] - 15s 249ms/step - accuracy: 0.7400 - loss: 0.5345 - val_accuracy: 0.7396 - val_loss: 0.4495
Epoch 3/5
62/62 [==============================] - 17s 273ms/step - accuracy: 0.8152 - loss: 0.4666 - val_accuracy: 0.8125 - val_loss: 0.4123
Epoch 4/5
62/62 [==============================] - 15s 250ms/step - accuracy: 0.8800 - loss: 0.3946 - val_accuracy: 0.7917 - val_loss: 0.4052
Epoch 5/5
62/62 [==============================] - 15s 249ms/step - accuracy: 0.9033 - loss: 0.3048 - val_accuracy: 0.8333 - val_loss: 0.3883


In [12]:
# BERT BS8 Pretrained

with strategy.scope():
    model_tf = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")

    if True:
        # Reinitialize Pos Embeddings - This is necessary even on GPU if it throws no error as

        # On tf.gather:
        # Caution: On CPU, if an out of bound index is found, an error is raised. 
        # On GPU, if an out of bound index is found, a 0 is stored in the corresponding output value.

        from transformers.modeling_tf_utils import get_initializer

        config = model_tf.config
        model_tf.bert.embeddings.position_embeddings = model_tf.bert.embeddings.add_weight(
                                                        name="embeddings",
                                                        shape=[2048, 768],
                                                        initializer=get_initializer(initializer_range=config.initializer_range),
                                                        )

    model_tf = WrapperFinetuning(model_tf)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model_tf.compile(optimizer=optimizer, metrics=['accuracy'])
    
model_tf.fit(train_ds, epochs=5, validation_data=val_ds)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.


Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5


62/62 [==============================] - ETA: 0s - accuracy: 0.5867 - loss: 0.6654WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


62/62 [==============================] - 86s 369ms/step - accuracy: 0.5868 - loss: 0.6654 - val_accuracy: 0.7292 - val_loss: 0.5951
Epoch 2/5
62/62 [==============================] - 15s 250ms/step - accuracy: 0.6497 - loss: 0.6247 - val_accuracy: 0.7604 - val_loss: 0.5461
Epoch 3/5
62/62 [==============================] - 15s 247ms/step - accuracy: 0.7366 - loss: 0.5483 - val_accuracy: 0.8021 - val_loss: 0.4614
Epoch 4/5
62/62 [==============================] - 15s 247ms/step - accuracy: 0.7436 - loss: 0.4750 - val_accuracy: 0.7500 - val_loss: 0.5288
Epoch 5/5
62/62 [==============================] - 15s 248ms/step - accuracy: 0.8457 - loss: 0.4279 - val_accuracy: 0.8229 - val_loss: 0.4072


In [13]:
# BERT BS8 NOT Pretrained

with strategy.scope():
    model_tf = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")

    config = BertConfig()
    config.max_position_embeddings = 2048
    model_tf = TFBertForSequenceClassification(config)

    if False:
        # Reinitialize Pos Embeddings - This is necessary even on GPU if it throws no error as

        # On tf.gather:
        # Caution: On CPU, if an out of bound index is found, an error is raised. 
        # On GPU, if an out of bound index is found, a 0 is stored in the corresponding output value.

        from transformers.modeling_tf_utils import get_initializer

        config = model_tf.config
        model_tf.bert.embeddings.position_embeddings = model_tf.bert.embeddings.add_weight(
                                                        name="embeddings",
                                                        shape=[2048, 768],
                                                        initializer=get_initializer(initializer_range=config.initializer_range),
                                                        )

    model_tf = WrapperFinetuning(model_tf)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model_tf.compile(optimizer=optimizer, metrics=['accuracy'])
    
model_tf.fit(train_ds, epochs=5, validation_data=val_ds)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.


Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5


62/62 [==============================] - ETA: 0s - accuracy: 0.5064 - loss: 0.7363WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


62/62 [==============================] - 86s 367ms/step - accuracy: 0.5065 - loss: 0.7363 - val_accuracy: 0.7604 - val_loss: 0.5575
Epoch 2/5
62/62 [==============================] - 15s 247ms/step - accuracy: 0.5972 - loss: 0.6919 - val_accuracy: 0.7604 - val_loss: 0.6682
Epoch 3/5
62/62 [==============================] - 16s 260ms/step - accuracy: 0.5919 - loss: 0.6795 - val_accuracy: 0.7604 - val_loss: 0.5852
Epoch 4/5
62/62 [==============================] - 15s 249ms/step - accuracy: 0.6174 - loss: 0.6903 - val_accuracy: 0.7604 - val_loss: 0.6149
Epoch 5/5
62/62 [==============================] - 15s 248ms/step - accuracy: 0.6272 - loss: 0.6827 - val_accuracy: 0.2396 - val_loss: 0.7308
